# Step 1: Crawling

Get posts, post meta and comments from facebook fan pages

In [1]:
import sys, os
import time
import re
import json

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

import numpy as np
import pandas as pd

In [ ]:
driver_file = './src/chorme_driver/chromedriver' # point to the binary file where you store the chrome driver
email = 'your_facebook_account' # your facebook account
password = 'your_facebook_password'# your facebook password

In [ ]:
# the fanpage name we want to search
fanpage_name_list = ['ETtoday','YahooTWNews','appledaily.tw','news.ebc','setnews','tvbsfb','tvbsfb']
# keywords for the posts we're search within the fanpage
keyword_list = ['三倍券','振興券','消費券']

### Notes about xpath
WebElement foo = driver.findElement(By.xpath("//*[@title='" + title + "']"));
WebElement bar = foo.findElement(By.xpath(".//img"));
The leading dot in the second xpath makes all the difference, as it will search only for child elements of foo.

In [2]:
# login driver: go to facebook login page and type in account and password
class loginDriver:
    def __init__(self, driver_path):
        options = Options()
        options.add_argument("--disable-notifications")
        self.driver = webdriver.Chrome(driver_path, options=options)
    def login(self, email, password):
        self.driver.get("https://www.facebook.com/")
        time.sleep(5)
        email_elem = self.driver.find_element_by_id("email")
        password_elem = self.driver.find_element_by_id("pass")
        email_elem.send_keys(email)
        password_elem.send_keys(password)
        password_elem.submit()
        time.sleep(5)
        return self.driver

In [3]:
# fbClawer, it could 
# (1) go the the given fan page, use the searching feature on facebook to find posts according to keyword 
# (2) get the post contents,meta data, reactions, comments  
# yea I know the code is a mess. 
# While facebook may change layout structure from time to time, it's understandable the code won't last long
class fbClawer:
    def __init__(self, driver):
        self.driver = driver
    
    def elem_click(self, elem):
        webdriver.ActionChains(self.driver).move_to_element(elem).click(elem).perform()
        time.sleep(np.random.rand()*2 + 2)
    
    def find_posts(self, fanpage_name, keyword, scroll_times):
        url = f'https://www.facebook.com/{fanpage_name}/'
        post_link_list = []
        self.driver.get(url)
        time.sleep(5)
        search_button_element = self.driver.find_element_by_xpath('//div[@aria-label="Search"]')
        self.elem_click(search_button_element)

        search_input_elem = self.driver.find_element_by_xpath('//input[@placeholder="Search this Page"]')
        search_input_elem.send_keys(keyword)
        search_input_elem.send_keys(u'\ue006') # press "return" key

        for i in range(scroll_times):
            self.driver.execute_script('window.scrollTo(10, document.body.scrollHeight);')
            WebDriverWait(self.driver, 10)
            time.sleep(np.random.randint(3)+2)

        links_elem_list = self.driver.find_elements_by_xpath('//div[@role="feed"]//a')

        for link in links_elem_list:
            href = link.get_attribute("href") # innerHTML, outerHTML could get raw html
            if href and 'posts' in href and url in href:
                href = href.split("?")[0]
                post_link_list.append(href)
        return list(set(post_link_list))
    
    def get_post_content(self, url):
        print(url)
        self.driver.get(url)
        time.sleep(5)
        re_dict = {'post_url':url}
        
        # the post is located under
        post_elem_xpath = '//div[@aria-labelledby and @role="article"]'
        post_elem = self.driver.find_elements_by_xpath(post_elem_xpath)[0]

        # post time
        posttime_elem_list = self.driver.find_elements_by_xpath('//div[@hidden="true"]//span')
        posttime = [i.get_attribute("innerHTML") for i in posttime_elem_list if not "Sponsored" in i.get_attribute("innerHTML")][0]
        re_dict['post_time'] = posttime

        # post content
        post_content_elem = post_elem.find_element_by_xpath('.//div[@data-ad-comet-preview="message"]')
        re_dict['post_content'] = post_content_elem.text

        # post referlink text
        post_referlink_elem_list = post_elem.find_elements_by_xpath('.//div[contains(@id,"jsc")]//a[@rel="nofollow noopener" and @role="link" and @href]//span[@dir="auto"]')
        if len(post_referlink_elem_list)>1:
            referlink_text = post_referlink_elem_list[1].text.replace("\u3000","")
            re_dict['link_title'] = referlink_text
        else:
            re_dict['link_title'] = ''

    #     reaction stats
        total_reaction = post_elem.find_element_by_xpath('.//span[@aria-hidden="true"]/span').text

        reaction_stat_link_elem = post_elem.find_element_by_xpath('.//span[@aria-label="See who reacted to this"]')
        reaction_stat_elem_list = reaction_stat_link_elem.find_elements_by_xpath('.//div')
        re_dict.update({'reaction_stat':
                        {'total':total_reaction,
                         'top_3_reaction':[i.get_attribute('aria-label') for i in reaction_stat_elem_list]
                        }})

        # comment
        # 1 - change showing comment to all comment
        try:
            change_showing_comment_element = post_elem.find_element_by_xpath('.//div[@role="button"]//*[contains(text(),\'Most Relevant\')]')
        except:
            self.dumpdata(url, re_dict)
            return
        self.elem_click(change_showing_comment_element)
        allcomment_element = post_elem.find_element_by_xpath('//div[@tabindex="-1"]//div[@data-pagelet="root"]//div[@role="menu"]//div[@role="menuitem"]//span[@dir="auto" and contains(text(),"All Comments")]')
        self.elem_click(allcomment_element)

        time.sleep(5)
        # 2 - expanding all show more comments

        comment_expand_times = 0
        while True:
            temp = post_elem.find_elements_by_xpath('.//div[@data-visualcompletion]')
            # broute force to find main post section
            for elem_ind in range(len(temp)):
                if re.search("Write\ a\ comment", temp[elem_ind].text):
                    break
            comment_sec_elem = temp[elem_ind]

            has_match_morecomment = re.search("View\ (\d+\ )?more\ comments", comment_sec_elem.text)
            if has_match_morecomment:
                print(f"expanding comments {comment_expand_times}")
                comment_expand_times+=1
                more_comment_button_elem = comment_sec_elem.find_element_by_xpath('.//*[contains(text(),"'+has_match_morecomment.group(0)+'")]')
                self.elem_click(more_comment_button_elem)
                time.sleep(10)
            else:
                break

        comment_elem_list = comment_sec_elem.find_elements_by_xpath(".//ul")[1].find_elements_by_xpath(".//li")
        print(f"possilbe have {len(comment_elem_list)/5} comments")

        # 3 - expand all comments and reply, get the contents
        comment_list = []
        for elem in comment_elem_list:
            comment_dict = {}
            # sample comment 
            # 'Chun Steel\n留個百年做紀念吧……\n15\nLike\n · Reply · See Translation · 2h'
            comment_elem = elem.find_elements_by_xpath('./div')

            if(len(comment_elem)!=2):
                continue
                
            # "see more" click out for main comment
            comment_main_elem = comment_elem[0]
            try:
                see_more_button_elem = comment_main_elem.find_element_by_xpath('.//div[@role="button" and contains(text(),"See More")]')
                self.elem_click(see_more_button_elem)
            except:
                pass
            comment_dict['main'] = comment_main_elem.text.split("\nLike")[0]

            # replies
            comment_reply_elem = comment_elem[1]
            if len(comment_reply_elem.text)>0 and re.search("\d+\ Reply", comment_reply_elem.text) or re.search("\d+\ Replies", comment_reply_elem.text):
                self.elem_click(comment_reply_elem)
                flag = True
                while flag:
                    flag = False
                    try:
                        possible_more_reply_elem_list = comment_reply_elem.find_elements_by_xpath('.//span[contains(text(),"View")]')
                        for i in range(len(possible_more_reply_elem_list)):
                            if not "Hide" in possible_more_reply_elem_list[i].text:
                                flag = True
                                self.elem_click(possible_more_reply_elem_list[i])
                    except Exception as e:
                        flag = False

                reply_elem_list = comment_reply_elem.find_elements_by_xpath('.//div[contains(@aria-label,"Reply")]')
                for reply_elem in reply_elem_list:
                    try:
                        see_more_button_elem = reply_elem.find_element_by_xpath('.//div[@role="button" and contains(text(),"See More")]')
                        self.elem_click(see_more_button_elem)
                    except:
                        pass
                comment_dict['replies'] = [reply.text.split("\nLike")[0] for reply in reply_elem_list[:-1]]

            else:
                comment_dict['replies'] = []
            comment_list.append(comment_dict)
        re_dict.update({'comments':comment_list})
        
        self.dumpdata(url, re_dict)
        return
        
    def dumpdata(self, url, re_dict):
        splitted_url = url.split("/posts/")
        with open("./data/"+splitted_url[0].split("/")[-1]+'-'+splitted_url[1], 'w') as outfile:
            json.dump(re_dict, outfile, ensure_ascii=False)

In [6]:
mydriver = loginDriver(driver_path=driver_file).login(email=email, password=password)
crawler = fbClawer(mydriver)

# Get all the posts links

In [1]:
# scroll_times = 20

# postlinks = []
# for fanpage_name in fanpage_name_list:
#     for keyword in keyword_list:
#         add_postlinks = crawler.find_posts(fanpage_name, keyword, scroll_times = scroll_times)
#         print(f'{fanpage_name} {keyword} - {len(add_postlinks)}')
#         postlinks+=add_postlinks
# postlinks = set(postlinks)

In [ ]:
# save all post links
postlink_ser = pd.Series(list(postlinks))
postlink_ser.to_csv("./data/post_links.csv",index=None)

**note**
Total we would get 600+ posts related to the topic.   
We still need to do some manuaul work here to check each post to exclude un-related posts. Save them into "data/exclude_link.txt"

# For each post, get the post content and comments and its reaction
This may takes a while. 

In [9]:
postlink_ser = pd.read_csv("./data/post_links.csv")['0']
skip_url_list = open("./data/exclude_link.txt",'r').readlines()# links we don't want to crawl
skip_url_list = [i.replace("\n","") for i in skip_url_list]
f.close()

postlink_ser = postlink_ser[np.logical_not(postlink_ser.isin(skip_url_list))].reset_index(drop=True)
print(len(postlink_ser))
postlink_ser = postlink_ser.sample(frac=1)

683


In [10]:
for link in postlink_ser.tolist():
    splitted_link = link.split("/posts/")
    if splitted_link[0].split("/")[-1]+"-"+splitted_link[-1] in os.listdir("./data/"):
        continue
    else:
        try:
            crawler.get_post_content(link)
        except Exception as e:
            print(f"error {link}")
            print(e)
            f = open("errlog.txt", "a")
            f.write(link+"\n")
            f.close()

https://www.facebook.com/YahooTWNews/posts/3476696202379321
expanding comments 0
expanding comments 1
possilbe have 83.0 comments
https://www.facebook.com/YahooTWNews/posts/3464992330216375
expanding comments 0
expanding comments 1
possilbe have 59.6 comments
https://www.facebook.com/news.ebc/posts/4693202120714842
expanding comments 0
expanding comments 1
expanding comments 2
expanding comments 3
expanding comments 4
expanding comments 5
possilbe have 298.0 comments
https://www.facebook.com/YahooTWNews/posts/3607031006012506
expanding comments 0
expanding comments 1
expanding comments 2
expanding comments 3
possilbe have 160.6 comments
https://www.facebook.com/ETtoday/posts/3437448309624642
expanding comments 0
expanding comments 1
expanding comments 2
expanding comments 3
expanding comments 4
expanding comments 5
expanding comments 6
expanding comments 7
expanding comments 8
error https://www.facebook.com/ETtoday/posts/3437448309624642
https://www.facebook.com/appledaily.tw/posts/101